In [48]:
import pandas as pd
import re
from datetime import datetime
from dateutil import parser
import numpy as np

In [49]:
df = pd.read_csv('films_financials.csv', low_memory=False)

In [50]:
df.head()

,title,top_cast,director,writer,country_of_origin,language,release_date,production_company,filming_locations,aspect_ratio,budget_local_currency,gross_us_canada,opening_weekend_us_canada_usd,opening_weekend_date,gross_worldwide,movie_url
0,BoJack Horseman,"Will Arnett,Amy Sedaris,Alison Brie,Aaron Paul...",Raphael Bob-Waksberg,NaN,United States,English,"August 22, 2014 (United States)","Tornante Company,ShadowMachine,Netflix",NaN,16:9 HD,NaN,NaN,NaN,NaN,NaN,https://www.imdb.com/title/tt3398228/?ref_=kw_...
1,The Thomas Crown Affair,"Pierce Brosnan,Rene Russo,Denis Leary,Ben Gazz...",John McTiernan,Alan Trustman,United States,English,"August 6, 1999 (United States)","United Artists,Irish DreamTime,Metro-Goldwyn-M...","Martinique, Départements d'Outre-Mer, France",2.35 : 1,"$48,000,000 (estimated)","$69,305,181","$14,600,719","Aug 8, 1999","$124,305,181",https://www.imdb.com/title/tt0155267/?ref_=kw_...
2,The Little Mermaid,"Halle Bailey,Jonah Hauer-King,Melissa McCarthy...",Rob Marshall,David Magee,United States,English,"May 26, 2023 (United States)","Walt Disney Pictures,Lucamar Productions,Marc ...","Sardinia, Italy",2.39 : 1,NaN,NaN,NaN,NaN,NaN,https://www.imdb.com/title/tt5971474/?ref_=kw_...
3,The Silence of the Lambs,"Jodie Foster,Anthony Hopkins,Lawrence A. Bonne...",Jonathan Demme,Thomas Harris,United States,English,"February 14, 1991 (United States)","Strong Heart/Demme Production,Orion Pictures","Bimini Airport, South Bimini Island, Bimini Is...",1.85 : 1,"$19,000,000 (estimated)","$130,742,922","$13,766,814","Feb 18, 1991","$272,742,922",https://www.imdb.com/title/tt0102926/?ref_=kw_...
4,Mr. Nice Guy,"Jackie Chan,Richard Norton,Miki Lee,Karen McLy...",Sammo Kam-Bo Hung,Fibe Ma,Hong Kong,English,"March 20, 1998 (United States)","GH Pictures,Golden Harvest Company,Golden Harv...","Chinatown Mall, Fortitude Valley, Queensland, ...",2.35 : 1,NaN,"$12,716,953","$5,250,704","Mar 22, 1998","$18,814,720",https://www.imdb.com/title/tt0117786/?ref_=kw_...


In [4]:
df.loc[:, 'movie_id'] = df.loc[:,'movie_url'].str.extract('([t]{2}\d+)', expand=False)

In [5]:
df.shape

(1260, 17)

In [6]:
list_columns = ['top_cast', 'director', 'writer', 'country_of_origin', 'production_company']

In [7]:
def table_creator(df, columns): 
    dfs = []
    for column in list_columns: 
        temp_df = df[['movie_id', column]].copy()
        temp_df.loc[:, column] = temp_df.loc[:, column].str.split(',')
        temp_df = temp_df.explode(column)
        temp_df.set_index('movie_id', inplace=True)
        dfs.append(temp_df)
    return dfs      

In [8]:
df_copy = df.copy()

In [9]:
list_of_dataframes = table_creator(df, list_columns)
for df in list_of_dataframes: 
    display(df.head())

,top_cast
movie_id,
tt3398228,Will Arnett
tt3398228,Amy Sedaris
tt3398228,Alison Brie
tt3398228,Aaron Paul
tt3398228,Paul F. Tompkins


,director
movie_id,
tt3398228,Raphael Bob-Waksberg
tt0155267,John McTiernan
tt5971474,Rob Marshall
tt0102926,Jonathan Demme
tt0117786,Sammo Kam-Bo Hung


,writer
movie_id,
tt3398228,NaN
tt0155267,Alan Trustman
tt5971474,David Magee
tt0102926,Thomas Harris
tt0117786,Fibe Ma


,country_of_origin
movie_id,
tt3398228,United States
tt0155267,United States
tt5971474,United States
tt0102926,United States
tt0117786,Hong Kong


,production_company
movie_id,
tt3398228,Tornante Company
tt3398228,ShadowMachine
tt3398228,Netflix
tt0155267,United Artists
tt0155267,Irish DreamTime


In [10]:
cast = list_of_dataframes[0]
director = list_of_dataframes[1]
writer = list_of_dataframes[2]
country_of_origin = list_of_dataframes[3]
production_company = list_of_dataframes[4]

In [11]:
df_copy.head()

,title,top_cast,director,writer,country_of_origin,language,release_date,production_company,filming_locations,aspect_ratio,budget_local_currency,gross_us_canada,opening_weekend_us_canada_usd,opening_weekend_date,gross_worldwide,movie_url,movie_id
0,BoJack Horseman,"Will Arnett,Amy Sedaris,Alison Brie,Aaron Paul...",Raphael Bob-Waksberg,NaN,United States,English,"August 22, 2014 (United States)","Tornante Company,ShadowMachine,Netflix",NaN,16:9 HD,NaN,NaN,NaN,NaN,NaN,https://www.imdb.com/title/tt3398228/?ref_=kw_...,tt3398228
1,The Thomas Crown Affair,"Pierce Brosnan,Rene Russo,Denis Leary,Ben Gazz...",John McTiernan,Alan Trustman,United States,English,"August 6, 1999 (United States)","United Artists,Irish DreamTime,Metro-Goldwyn-M...","Martinique, Départements d'Outre-Mer, France",2.35 : 1,"$48,000,000 (estimated)","$69,305,181","$14,600,719","Aug 8, 1999","$124,305,181",https://www.imdb.com/title/tt0155267/?ref_=kw_...,tt0155267
2,The Little Mermaid,"Halle Bailey,Jonah Hauer-King,Melissa McCarthy...",Rob Marshall,David Magee,United States,English,"May 26, 2023 (United States)","Walt Disney Pictures,Lucamar Productions,Marc ...","Sardinia, Italy",2.39 : 1,NaN,NaN,NaN,NaN,NaN,https://www.imdb.com/title/tt5971474/?ref_=kw_...,tt5971474
3,The Silence of the Lambs,"Jodie Foster,Anthony Hopkins,Lawrence A. Bonne...",Jonathan Demme,Thomas Harris,United States,English,"February 14, 1991 (United States)","Strong Heart/Demme Production,Orion Pictures","Bimini Airport, South Bimini Island, Bimini Is...",1.85 : 1,"$19,000,000 (estimated)","$130,742,922","$13,766,814","Feb 18, 1991","$272,742,922",https://www.imdb.com/title/tt0102926/?ref_=kw_...,tt0102926
4,Mr. Nice Guy,"Jackie Chan,Richard Norton,Miki Lee,Karen McLy...",Sammo Kam-Bo Hung,Fibe Ma,Hong Kong,English,"March 20, 1998 (United States)","GH Pictures,Golden Harvest Company,Golden Harv...","Chinatown Mall, Fortitude Valley, Queensland, ...",2.35 : 1,NaN,"$12,716,953","$5,250,704","Mar 22, 1998","$18,814,720",https://www.imdb.com/title/tt0117786/?ref_=kw_...,tt0117786


In [12]:
df_copy.drop(['top_cast', 'director', 'writer', 'country_of_origin', 'production_company'], axis=1, inplace=True)

In [13]:
digit_columns = ['budget_local_currency', 'gross_us_canada', 'opening_weekend_us_canada_usd', 'gross_worldwide']

In [14]:
def comma_cleaner(df, columns): 
    for column in columns: 
        df[column].replace(',','', regex=True, inplace=True)
    return df

In [15]:
df2 = comma_cleaner(df_copy, digit_columns)

In [16]:
df2.loc[:, 'local_currency'] = df2.loc[:,'budget_local_currency'].str.extract('(\D+)', expand=False)

In [17]:
df2.loc[:, 'local_currency_budget'] = df2.loc[:,'budget_local_currency'].str.extract('(\d+)', expand=False)

In [18]:
df2.drop('budget_local_currency', axis=1, inplace=True)

In [19]:
df2['release_country'] = df2['release_date'].astype(str).apply(lambda x: x.split('(')[-1]).apply(lambda x: x.strip(')'))

In [20]:
df2['release_date'] = df2['release_date'].astype(str).apply(lambda x: x.split('(')[0])

In [21]:
df2['release_date'] = df2['release_date'].astype(str).apply(lambda x: x.replace(',', '')).apply(lambda x: x.replace('  ', ' '))

In [22]:
df2 = df2[[(len(x) > 7) for x in df2['release_date']]].copy()

In [23]:
df2 = pd.DataFrame(df2)

pd.to_datetime('04SEP12:00:00:00', format='%d%b%y:%H:%M:%S')

In [24]:
def month_replacement(row):
    months = {
        "January": '01',
        "February": '02',
        "March": '03',
        "April": '04',
        "May": '05',
        "June": '06',
        "July": '07',
        "August": '08',
        "September": '09',
        "October": '10',
        "November": '11',
        "December": '12'
    }
    old_date = row['release_date']
    row['release_date'] = ''.join([months.get(i, i) for i in old_date.split()])
    return row

In [25]:
df2 = df2.apply(month_replacement, axis=1)

In [26]:
df2['release_date'] = pd.to_datetime(df2['release_date'], format='%m%d%Y', errors='coerce')

In [27]:
df2['opening_weekend_date'] = df2['opening_weekend_date'].astype(str).apply(lambda x: x.replace(',', '')).apply(lambda x: x.replace('  ', ' '))

In [28]:
def short_month_replacement(row):
    months = {
        "Jan": '01',
        "Feb": '02',
        "Mar": '03',
        "Apr": '04',
        "May": '05',
        "Jun": '06',
        "Jul": '07',
        "Aug": '08',
        "Sep": '09',
        "Oct": '10',
        "Nov": '11',
        "Dec": '12'
    }
    old_date = row['opening_weekend_date']
    row['opening_weekend_date'] = ''.join([months.get(i, i) for i in old_date.split()])
    return row

In [29]:
df2 = df2.apply(short_month_replacement, axis=1)

In [30]:
df2['opening_weekend_date'] = pd.to_datetime(df2['opening_weekend_date'], format='%m%d%Y', errors='coerce')

In [31]:
categories = ['language', 'filming_locations', 'filming_locations', 'aspect_ratio', 'local_currency', 'release_country']

In [32]:
for column in categories: 
    df2[column].fillna('Not defined', inplace=True)

In [33]:
def currency_replacement(row):
    currencies = {
        '$': 'USD',
        '€': 'EUR',
        '£': 'GBP',
        'CA$': 'CAD',
        '₹': 'INR',
        'RUR': 'RUB'
    }
    old_currency = row['local_currency']
    row['local_currency'] = ' '.join(
        [currencies.get(i, i) for i in old_currency.split()])
    return row

In [34]:
df2 = df2.apply(currency_replacement, axis=1)

In [35]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1184 entries, 0 to 1259
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   title                          1184 non-null   object        
 1   language                       1184 non-null   object        
 2   release_date                   1163 non-null   datetime64[ns]
 3   filming_locations              1184 non-null   object        
 4   aspect_ratio                   1184 non-null   object        
 5   gross_us_canada                238 non-null    object        
 6   opening_weekend_us_canada_usd  225 non-null    object        
 7   opening_weekend_date           225 non-null    datetime64[ns]
 8   gross_worldwide                272 non-null    object        
 9   movie_url                      1184 non-null   object        
 10  movie_id                       1184 non-null   object        
 11  local_currency   

In [36]:
money_cols = ['gross_us_canada', 'opening_weekend_us_canada_usd', 'gross_worldwide', 'local_currency_budget']

In [37]:
for col in money_cols: 
    df2[col] = df2[col].str.replace('$', '', regex=True)
    df2[col] = df2[col].astype(float)

In [38]:
df2.head()

,title,language,release_date,filming_locations,aspect_ratio,gross_us_canada,opening_weekend_us_canada_usd,opening_weekend_date,gross_worldwide,movie_url,movie_id,local_currency,local_currency_budget,release_country
0,BoJack Horseman,English,2014-08-22,Not defined,16:9 HD,NaN,NaN,NaT,NaN,https://www.imdb.com/title/tt3398228/?ref_=kw_...,tt3398228,Not defined,NaN,United States
1,The Thomas Crown Affair,English,1999-08-06,"Martinique, Départements d'Outre-Mer, France",2.35 : 1,69305181.0,14600719.0,1999-08-08,124305181.0,https://www.imdb.com/title/tt0155267/?ref_=kw_...,tt0155267,USD,48000000.0,United States
2,The Little Mermaid,English,2023-05-26,"Sardinia, Italy",2.39 : 1,NaN,NaN,NaT,NaN,https://www.imdb.com/title/tt5971474/?ref_=kw_...,tt5971474,Not defined,NaN,United States
3,The Silence of the Lambs,English,1991-02-14,"Bimini Airport, South Bimini Island, Bimini Is...",1.85 : 1,130742922.0,13766814.0,1991-02-18,272742922.0,https://www.imdb.com/title/tt0102926/?ref_=kw_...,tt0102926,USD,19000000.0,United States
4,Mr. Nice Guy,English,1998-03-20,"Chinatown Mall, Fortitude Valley, Queensland, ...",2.35 : 1,12716953.0,5250704.0,1998-03-22,18814720.0,https://www.imdb.com/title/tt0117786/?ref_=kw_...,tt0117786,Not defined,NaN,United States


In [39]:
def filming_location_split(row):
    if row['filming_locations'] != 'Not defined': 
        location = row['filming_locations'].split(',')
        row['filming_country'] = location[-1]
        row['filming_locations'] = ', '.join(location[:-1])
    return row

In [40]:
df2 = df2.apply(filming_location_split, axis=1)

In [41]:
df2.head()

,aspect_ratio,filming_country,filming_locations,gross_us_canada,gross_worldwide,language,local_currency,local_currency_budget,movie_id,movie_url,opening_weekend_date,opening_weekend_us_canada_usd,release_country,release_date,title
0,16:9 HD,NaN,Not defined,NaN,NaN,English,Not defined,NaN,tt3398228,https://www.imdb.com/title/tt3398228/?ref_=kw_...,NaT,NaN,United States,2014-08-22,BoJack Horseman
1,2.35 : 1,France,"Martinique, Départements d'Outre-Mer",69305181.0,124305181.0,English,USD,48000000.0,tt0155267,https://www.imdb.com/title/tt0155267/?ref_=kw_...,1999-08-08,14600719.0,United States,1999-08-06,The Thomas Crown Affair
2,2.39 : 1,Italy,Sardinia,NaN,NaN,English,Not defined,NaN,tt5971474,https://www.imdb.com/title/tt5971474/?ref_=kw_...,NaT,NaN,United States,2023-05-26,The Little Mermaid
3,1.85 : 1,Bahamas,"Bimini Airport, South Bimini Island, Bimini ...",130742922.0,272742922.0,English,USD,19000000.0,tt0102926,https://www.imdb.com/title/tt0102926/?ref_=kw_...,1991-02-18,13766814.0,United States,1991-02-14,The Silence of the Lambs
4,2.35 : 1,Australia,"Chinatown Mall, Fortitude Valley, Queensland",12716953.0,18814720.0,English,Not defined,NaN,tt0117786,https://www.imdb.com/title/tt0117786/?ref_=kw_...,1998-03-22,5250704.0,United States,1998-03-20,Mr. Nice Guy


In [42]:
df2.columns

Index(['aspect_ratio', 'filming_country', 'filming_locations',
       'gross_us_canada', 'gross_worldwide', 'language', 'local_currency',
       'local_currency_budget', 'movie_id', 'movie_url',
       'opening_weekend_date', 'opening_weekend_us_canada_usd',
       'release_country', 'release_date', 'title'],
      dtype='object')

In [43]:
df2 = df2[['movie_id', 'title', 'filming_country', 'filming_locations',
          'language', 'release_country', 'release_date', 'aspect_ratio',
          'local_currency_budget', 'local_currency', 'opening_weekend_date',
          'opening_weekend_us_canada_usd', 'gross_us_canada',
          'gross_worldwide', 'movie_url']]

In [44]:
df2['filming_country'] = df2['filming_country'].fillna('Not defined')

In [45]:
df2.head()

,movie_id,title,filming_country,filming_locations,language,release_country,release_date,aspect_ratio,local_currency_budget,local_currency,opening_weekend_date,opening_weekend_us_canada_usd,gross_us_canada,gross_worldwide,movie_url
0,tt3398228,BoJack Horseman,Not defined,Not defined,English,United States,2014-08-22,16:9 HD,NaN,Not defined,NaT,NaN,NaN,NaN,https://www.imdb.com/title/tt3398228/?ref_=kw_...
1,tt0155267,The Thomas Crown Affair,France,"Martinique, Départements d'Outre-Mer",English,United States,1999-08-06,2.35 : 1,48000000.0,USD,1999-08-08,14600719.0,69305181.0,124305181.0,https://www.imdb.com/title/tt0155267/?ref_=kw_...
2,tt5971474,The Little Mermaid,Italy,Sardinia,English,United States,2023-05-26,2.39 : 1,NaN,Not defined,NaT,NaN,NaN,NaN,https://www.imdb.com/title/tt5971474/?ref_=kw_...
3,tt0102926,The Silence of the Lambs,Bahamas,"Bimini Airport, South Bimini Island, Bimini ...",English,United States,1991-02-14,1.85 : 1,19000000.0,USD,1991-02-18,13766814.0,130742922.0,272742922.0,https://www.imdb.com/title/tt0102926/?ref_=kw_...
4,tt0117786,Mr. Nice Guy,Australia,"Chinatown Mall, Fortitude Valley, Queensland",English,United States,1998-03-20,2.35 : 1,NaN,Not defined,1998-03-22,5250704.0,12716953.0,18814720.0,https://www.imdb.com/title/tt0117786/?ref_=kw_...


In [46]:
df2.to_csv('financial_data.csv')

In [47]:
cast.to_csv('top_cast.csv')
director.to_csv('directors.csv')
writer.to_csv('writers.csv')
country_of_origin.to_csv('countries_of_origin.csv')
production_company.to_csv('production_companies.csv')